In [ ]:
"""
GTFS-Based Transit Itinerary Reconstruction Algorithm

This notebook implements the GTFS-based route matching algorithm described in:
"Beyond Mode Detection: Reconstructing Detailed Transit Itineraries from 
Crowdsourced GPS Trajectories" (SpatialConnect '25)

The algorithm uses R-tree spatial indexing and Longest Common Subsequence (LCSS)
matching to identify specific transit routes, boarding/alighting stops, and 
reconstruct complete multi-modal transit itineraries from raw GPS trajectories.

Key Features:
- R-tree spatial indexing for efficient route candidate retrieval
- LCSS-based trajectory-to-route matching with ~80% accuracy
- Automatic staypoint detection and trip segmentation
- Stop identification for boarding/alighting locations
- Multi-modal itinerary reconstruction with transfer detection

Authors: Ojas Jagtap, Naman Awasthi, Mohit Jain, Saad Mohammad Abrar, 
         Vanessa Frías-Martínez
Affiliation: University of Maryland
"""

# Standard library imports
import os

# Third-party imports
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
from shapely.geometry import Point, LineString, shape
from shapely.ops import linemerge
from rtree import index
from tqdm import tqdm

# Domain-specific imports
import trackintel as ti
import transbigdata as tbd
from gtfs_functions import Feed

In [ ]:
# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def plot(seg_data):
    """
    Visualize reconstructed transit itinerary on an interactive Folium map.
    
    Creates a color-coded visualization where each trip leg is displayed with:
    - Solid lines for GPS trajectories
    - Dashed lines for matched GTFS routes
    - Circle markers for boarding/alighting stops
    
    Parameters
    ----------
    seg_data : list of dict
        List of trip segments, each containing:
        - seg_geom: GPS trajectory as LineString
        - route_geom: Matched GTFS route geometry (if matched)
        - start_stop_geom: Boarding stop location (if identified)
        - end_stop_geom: Alighting stop location (if identified)
    
    Returns
    -------
    folium.Map
        Interactive map with reconstructed itinerary visualization
        
    Notes
    -----
    Each segment is assigned a unique color from the viridis colormap.
    Walking segments (no matched route) are shown as solid lines only.
    """
    # Initialize map centered on data
    m = folium.Map()
    
    # Generate distinct colors for each segment
    colors = [to_hex(i) for i in plt.cm.viridis(np.linspace(0, 1, len(seg_data)))]
    
    # Plot each trip segment
    for i, seg in enumerate(seg_data):
        seg_geom = seg['seg_geom']
        route_id = seg['route_id']
        route_geom = seg['route_geom']
        start_stop_geom = seg['start_stop_geom']
        end_stop_geom = seg['end_stop_geom']

        # Plot GPS trajectory (solid line)
        folium.PolyLine(
            locations=list([coord[::-1] for coord in seg_geom.coords]), 
            weight=2, 
            color=colors[i]
        ).add_to(m)

        # If transit leg is matched, plot route and stops
        if start_stop_geom and end_stop_geom:
            # Plot matched GTFS route (dashed line)
            folium.PolyLine(
                locations=list([coord[::-1] for coord in route_geom.coords]), 
                weight=2, 
                color=colors[i], 
                dash_array='10'
            ).add_to(m)
            
            # Plot boarding stop
            folium.CircleMarker(
                location=[start_stop_geom.x, start_stop_geom.y], 
                radius=5, 
                color=colors[i]
            ).add_to(m)
            
            # Plot alighting stop
            folium.CircleMarker(
                location=[end_stop_geom.x, end_stop_geom.y], 
                radius=5, 
                color=colors[i]
            ).add_to(m)

    # Auto-fit map bounds to data
    m.fit_bounds(m.get_bounds())
    return m

In [ ]:
def gen_seg_df(seg_data):
    """
    Generate a summary DataFrame of reconstructed trip segments.
    
    Extracts key information from matched segments to create a human-readable
    summary table showing route names and boarding/alighting stops for each leg.
    
    Parameters
    ----------
    seg_data : list of dict
        List of trip segments with route and stop information
    
    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
        - route_name: Transit line name (None for walking segments)
        - start_stop_name: Boarding stop name
        - end_stop_name: Alighting stop name
        
    Notes
    -----
    This summary table provides a quick overview of the reconstructed itinerary,
    similar to what would appear in a trip planner interface.
    """
    seg_df = pd.DataFrame(
        seg_data, 
        columns=["route_name", "start_stop_name", "end_stop_name"]
    )
    return seg_df

In [ ]:
# ============================================================================
# TRAJECTORY PREPROCESSING PIPELINE
# ============================================================================

def generate_pfs(trip):
    """
    Preprocess raw GPS trajectory and segment into trip legs.
    
    This function implements the complete preprocessing pipeline:
    1. Data cleaning: Remove GPS drift and outliers
    2. Trajectory densification: Interpolate to uniform 10-second intervals
    3. Smoothing: Apply Kalman filter to reduce noise
    4. Staypoint detection: Identify stops using sliding window
    5. Trip segmentation: Split trajectory at staypoints into legs
    
    Parameters
    ----------
    trip : pd.DataFrame
        Raw GPS trajectory with columns:
        - timestamp: datetime of GPS point
        - tripIdPk: unique trip identifier
        - lat: latitude
        - lon: longitude
    
    Returns
    -------
    ti.Positionfixes
        Trackintel positionfixes GeoDataFrame with additional columns:
        - tripleg_id: identifier for each continuous movement segment
        - geometry: Point geometries for each GPS location
        
    Notes
    -----
    Preprocessing parameters (configurable):
    - Speed limit: 120 km/h (removes implausible speeds)
    - Distance limit: 1000 m (removes large jumps)
    - Angle limit: 30° (detects heading anomalies)
    - Time gap: 10 seconds (densification interval)
    - Kalman process noise: 0.5 (smoothing strength)
    - Staypoint radius: 100 m
    - Staypoint duration: 3 minutes
    
    These parameters are tuned for urban transit environments and typical
    GPS sampling rates (~2 points/minute).
    """
    # Sort by time and prepare column names for TransBigData
    trip = trip[['timestamp', 'tripIdPk', 'lat', 'lon']].sort_values(by='timestamp')
    
    trip_drift = trip.rename(columns={
        'tripIdPk': 'VehicleNum',
        'timestamp': 'Time',
        'lon': 'Lng',
        'lat': 'Lat'
    })
    
    # Step 1: Clean GPS drift and outliers
    # Removes points with implausible speeds (>120 km/h), large jumps (>1000 m),
    # or anomalous heading changes (>30°)
    cleaned = tbd.traj_clean_drift(
        data=trip_drift,
        col=['VehicleNum', 'Time', 'Lng', 'Lat'],
        method='twoside',
        speedlimit=120,    # km/h
        dislimit=1000,     # meters
        anglelimit=30      # degrees
    )    
    cleaned = cleaned.rename(columns={'VehicleNum': 'Vehicleid'})
    
    # Step 2: Densify trajectory to uniform 10-second intervals
    # Interpolates points to ensure consistent temporal resolution
    densified = tbd.traj_densify(
        data=cleaned,
        col=['Vehicleid', 'Time', 'Lng', 'Lat'],
        timegap=10  # seconds
    )
    densified = densified.dropna(subset=['Lat', 'Lng'])

    # Step 3: Smooth trajectory using Kalman filter
    # Reduces GPS noise while preserving true movement patterns
    smoothed = tbd.traj_smooth(
        data=densified,
        col=['Vehicleid', 'Time', 'Lng', 'Lat'],
        proj=False,  # Work in lat/lon coordinates
        process_noise_std=0.5,      # Lower = smoother trajectory
        measurement_noise_std=1     # GPS measurement uncertainty
    )
    
    # Restore original column names
    smoothed = smoothed.rename(columns={
        'Vehicleid': 'tripIdPk',
        'Time': 'timestamp',
        'Lng': 'lon',
        'Lat': 'lat'
    })
    smoothed = smoothed.sort_values(by='timestamp').reset_index(drop=True)
    smoothed.index.name = 'pfs_id'
    
    # Convert to GeoDataFrame with Point geometries
    gdf = gpd.GeoDataFrame(
        smoothed,
        geometry=gpd.points_from_xy(smoothed.lon, smoothed.lat),
        crs='epsg:4326'  # WGS84 coordinate system
    )
    gdf.index.name = 'pfs_id'
    
    # Convert to trackintel positionfixes format
    pfs = ti.io.read_positionfixes_gpd(
        gdf,
        tracked_at='timestamp',
        user_id='tripIdPk',
        geom_col='geometry',
        tz='UTC'
    )
    
    # Step 4: Detect staypoints (stops) using sliding window
    # A staypoint is identified when user stays within 100m for at least 3 minutes
    pfs, sp = pfs.generate_staypoints(
        method='sliding', 
        dist_threshold=100,   # meters
        time_threshold=3      # minutes
    )
    
    # Step 5: Segment trajectory into trip legs at staypoints
    # Each tripleg represents continuous movement between two stops
    if sp.shape[0] > 0:
        pfs, tpls = pfs.generate_triplegs(sp, method="overlap_staypoints")
    else:
        # No staypoints detected - treat entire trajectory as one leg
        pfs['tripleg_id'] = 0

    return pfs

In [ ]:
# ============================================================================
# R-TREE SPATIAL INDEX CONSTRUCTION
# ============================================================================

def gen_rtree():
    """
    Build R-tree spatial index from GTFS transit route geometries.
    
    Loads GTFS feeds for all transit services in the region and constructs an
    R-tree spatial index on route geometries. The R-tree enables efficient
    spatial queries to retrieve candidate routes near a GPS trajectory.
    
    Returns
    -------
    route_index : rtree.index.Index
        R-tree spatial index with bounding boxes of all GTFS routes
    gtfs_routes : list of dict
        List of route information dictionaries, each containing:
        - feed: GTFS Feed object
        - geometry: route shape as LineString or MultiLineString
        - name: human-readable route name
        - route_id: GTFS route identifier
        
    Notes
    -----
    GTFS feeds loaded (Baltimore region):
    - Commuter Bus
    - Light Rail
    - Local Bus
    - MARC commuter rail
    - Metro subway
    - Charm City Circulator (CCC)
    
    Each route's position in gtfs_routes corresponds to its index in the R-tree,
    allowing efficient lookup of route details after spatial query.
    
    The R-tree uses bounding boxes for fast spatial filtering. Typical query
    complexity is O(log n) where n is the number of routes.
    """
    # Define paths to GTFS feed files
    # These should be updated to match your local GTFS data location
    commuterbus_path = 'GTFS/mdotmta_gtfs_commuterbus.zip'
    lightrail_path = 'GTFS/mdotmta_gtfs_lightrail.zip'
    localbus_path = 'GTFS/mdotmta_gtfs_localbus.zip'
    marc_path = 'GTFS/mdotmta_gtfs_marc.zip'
    metro_path = 'GTFS/mdotmta_gtfs_metro.zip'
    ccc_path = 'GTFS/mdotmta_gtfs_ccc.zip'
    
    # Load all GTFS feeds
    feeds = [
        Feed(commuterbus_path),
        Feed(lightrail_path),
        Feed(localbus_path),
        Feed(marc_path),
        Feed(metro_path),
        Feed(ccc_path)
    ]
    
    # Extract route geometries and metadata
    gtfs_routes = []
    route_index = index.Index()
    
    for feed in feeds:
        # Iterate through all route shapes in this feed
        for _, row in feed.shapes.iterrows():
            route_name = 'None'
            shape_id = row['shape_id']
            
            # Look up route name from shape_id
            matching_trips = feed.trips[feed.trips['shape_id'] == shape_id]
            if not matching_trips.empty:
                route_id = matching_trips['route_id'].iloc[0]
                mode_routes = feed.routes[feed.routes['route_id'] == route_id]
                if not mode_routes.empty:
                    route_name = mode_routes['route_long_name'].iloc[0]
    
            # Store route information
            gtfs_routes.append({
                "feed": feed, 
                "geometry": row["geometry"], 
                "name": route_name, 
                "route_id": route_id
            })
    
    # Build R-tree spatial index
    # Each route is indexed by its bounding box (minx, miny, maxx, maxy)
    for i, route in enumerate(gtfs_routes):
        route_index.insert(i, route["geometry"].bounds)
    
    return route_index, gtfs_routes

In [ ]:
# ============================================================================
# TRAJECTORY MATCHING ALGORITHMS
# ============================================================================

def lcss_2d(seq1, seq2, epsilon=0.0005):
    """
    Calculate 2D Longest Common Subsequence (LCSS) between two point sequences.
    
    LCSS measures trajectory similarity by finding the longest subsequence of
    points that match between two trajectories, where points are considered a
    match if they are within epsilon distance of each other.
    
    Unlike Dynamic Time Warping (DTW), LCSS is robust to outliers and temporal
    misalignment. It allows gaps in the matching sequence, making it suitable
    for noisy GPS data where some points may be erroneous.
    
    Parameters
    ----------
    seq1 : list of tuple
        First point sequence as [(x1, y1), (x2, y2), ...]
    seq2 : list of tuple
        Second point sequence as [(x1, y1), (x2, y2), ...]
    epsilon : float, optional
        Distance threshold for considering two points a match (default: 0.0005)
        In decimal degrees, ~0.0005° ≈ 50m at mid-latitudes
    
    Returns
    -------
    int
        Length of longest common subsequence (number of matching points)
        
    Notes
    -----
    Time complexity: O(n * m) where n, m are sequence lengths
    Space complexity: O(n * m)
    
    The algorithm uses dynamic programming with a 2D table L where:
    - L[i][j] = length of LCSS between seq1[0:i] and seq2[0:j]
    
    Algorithm:
    - If points i and j match (distance ≤ epsilon): L[i][j] = 1 + L[i-1][j-1]
    - Otherwise: L[i][j] = max(L[i-1][j], L[i][j-1])
    
    References
    ----------
    Vlachos, M., Kollios, G., & Gunopulos, D. (2002). "Discovering similar
    multidimensional trajectories." In Proceedings of ICDE.
    """
    n, m = len(seq1), len(seq2)
    
    # Initialize DP table: (n+1) x (m+1)
    L = np.zeros((n+1, m+1), dtype=int)
    
    # Fill DP table
    for i in range(1, n+1):
        for j in range(1, m+1):
            p1 = seq1[i-1]
            p2 = seq2[j-1]
            
            # Calculate Euclidean distance between points
            dist = np.linalg.norm(np.array(p1) - np.array(p2))
            
            if dist <= epsilon:
                # Points match - extend the LCSS
                L[i, j] = 1 + L[i-1, j-1]
            else:
                # No match - take maximum from skipping one point
                L[i, j] = max(L[i-1, j], L[i, j-1])
    
    return L[n, m]

In [ ]:
def match_leg_to_route_lcss(seg_geom, gtfs_routes, route_index, epsilon=0.0005):
    """
    Match a GPS trajectory segment to the best-fitting GTFS route using LCSS.
    
    This is the core route matching algorithm. It uses spatial indexing to
    efficiently retrieve candidate routes, then applies LCSS to find the route
    that best matches the GPS trajectory.
    
    Algorithm steps:
    1. Query R-tree to get candidate routes overlapping segment's bounding box
    2. For each candidate, calculate LCSS similarity score
    3. Select route with highest LCSS score as best match
    
    Parameters
    ----------
    seg_geom : shapely.geometry.LineString
        GPS trajectory segment to match
    gtfs_routes : list of dict
        List of all GTFS routes with geometries
    route_index : rtree.index.Index
        R-tree spatial index on route bounding boxes
    epsilon : float, optional
        LCSS distance threshold (default: 0.0005 ≈ 50m)
    
    Returns
    -------
    best_route : int or None
        Index of best matching route in gtfs_routes (None if no match)
    best_route_id : str or None
        GTFS route_id of best match
    best_feed : Feed or None
        GTFS Feed object containing the best match
    best_route_geom : LineString or None
        Geometry of best matching route
        
    Notes
    -----
    The R-tree dramatically reduces the search space:
    - Without R-tree: Must compare against all ~1000+ routes
    - With R-tree: Typically only 10-50 candidate routes to check
    
    LCSS scoring favors routes where the GPS trajectory stays close to the
    route geometry for a long continuous sequence. This naturally handles:
    - GPS noise (small deviations don't break the match)
    - Brief signal loss (gaps in GPS data)
    - Stops at transit stations (trajectory deviates to platform then returns)
    
    A higher LCSS score indicates better alignment between the GPS trajectory
    and the route shape.
    """
    # Extract coordinates from GPS trajectory
    seg_coords = list(seg_geom.coords)
    
    # Initialize best match tracking
    best_route = None
    best_route_id = None
    best_feed = None
    best_route_geom = None
    best_lcss_val = -1
    
    # Step 1: Query R-tree for candidate routes
    # Returns indices of routes whose bounding boxes intersect segment's bbox
    seg_bounds = seg_geom.bounds
    candidate_routes = list(route_index.intersection(seg_bounds))
    
    # Step 2: Evaluate each candidate route using LCSS
    for j in candidate_routes:
        route = gtfs_routes[j]
        route_geom = route["geometry"]
        
        # Handle MultiLineString geometries (routes with multiple segments)
        if route_geom.geom_type == 'MultiLineString':
            route_coords = []
            for line in route_geom:
                route_coords.extend(line.coords)
        else:
            route_coords = list(route_geom.coords)
        
        # Calculate LCSS similarity score
        current_lcss = lcss_2d(seg_coords, route_coords, epsilon=epsilon)

        # Update best match if this route scores higher
        if current_lcss > best_lcss_val:
            best_lcss_val = current_lcss
            best_route = j
            best_route_id = route["route_id"]
            best_feed = route["feed"]
            best_route_geom = route_geom
            
    return best_route, best_route_id, best_feed, best_route_geom

In [ ]:
def match_all_legs(pfs, gtfs_routes, route_index):
    """
    Match all trip legs in a trajectory to their respective transit routes.
    
    Processes each trip leg (continuous movement segment between staypoints)
    and attempts to match it to a GTFS route. Walking segments and unmatched
    legs are preserved in the output with None values.
    
    Parameters
    ----------
    pfs : ti.Positionfixes
        Trackintel positionfixes with tripleg_id column
    gtfs_routes : list of dict
        List of all GTFS routes with geometries
    route_index : rtree.index.Index
        R-tree spatial index on route bounding boxes
    
    Returns
    -------
    list of dict
        Matched routes for each trip leg, containing:
        - route_name: Transit line name (None if no match)
        - route_id: GTFS route identifier (None if no match)
        - feed: GTFS Feed object (None if no match)
        - seg_geom: GPS trajectory segment as LineString
        - route_geom: Matched GTFS route geometry (None if no match)
        
    Notes
    -----
    Legs with fewer than 2 GPS points are skipped (cannot form a LineString).
    
    No match (route_name = None) indicates either:
    1. Walking segment (expected)
    2. GPS data quality too poor for matching
    3. Trip on a route not in GTFS data (e.g., rideshare, unofficial route)
    """
    matched_routes = []

    # Process each trip leg independently
    for tripleg_id, tpl in pfs.groupby('tripleg_id'):
        # Skip legs with insufficient points
        if len(tpl['geometry']) < 2:
            continue
            
        # Create LineString from GPS points
        seg_geom = LineString(tpl['geometry'])
        
        # Match segment to best GTFS route
        best_route, best_route_id, best_feed, best_route_geom = match_leg_to_route_lcss(
            seg_geom, gtfs_routes, route_index
        )
        
        # Store matched route information
        matched_routes.append({
            "route_name": gtfs_routes[best_route]['name'] if best_route else None,
            "route_id": best_route_id,
            "feed": best_feed,
            "seg_geom": seg_geom,
            "route_geom": best_route_geom
        })

    return matched_routes

In [ ]:
# ============================================================================
# STOP IDENTIFICATION
# ============================================================================

def find_closest_stop(coords, stops, seg_geom):
    """
    Find the closest transit stop to a GPS coordinate along a route segment.
    
    Identifies the nearest stop from a set of candidate stops, with the
    constraint that the stop must lie on (or very near) the GPS trajectory
    segment. This ensures we select actual boarding/alighting stops rather
    than nearby stops on parallel routes.
    
    Parameters
    ----------
    coords : tuple
        GPS coordinates (lon, lat) to match
    stops : pd.DataFrame
        Candidate stops with columns:
        - stop_id: GTFS stop identifier
        - stop_name: Human-readable stop name
        - stop_lat: Stop latitude
        - stop_lon: Stop longitude
    seg_geom : shapely.geometry.LineString
        GPS trajectory segment (for filtering stops)
    
    Returns
    -------
    stop_name : str or None
        Name of closest stop, or None if no valid stop found
    stop_id : str or None
        GTFS stop_id, or None if no valid stop found
    stop_geom : Point or None
        Stop geometry as Point(lat, lon), or None if no valid stop found
        
    Notes
    -----
    The algorithm:
    1. Calculates distance from input coords to all candidate stops
    2. Filters to stops that lie on the GPS trajectory segment
    3. Returns the closest stop among filtered candidates
    
    This two-stage filtering prevents errors in dense transit networks where
    multiple routes have stops at the same intersection. We ensure the stop
    actually appears along the user's GPS trajectory.
    """
    trip_point = Point(coords)
    
    # Calculate distance from trip point to each candidate stop
    stops['distance'] = stops.apply(
        lambda stop: trip_point.distance(Point(stop['stop_lon'], stop['stop_lat'])),
        axis=1
    )
    
    # Filter to stops that lie on the GPS trajectory segment
    # This ensures we select stops the user actually passed, not nearby stops
    stops_on_seg = stops[stops.apply(
        lambda stop: seg_geom.contains(Point(stop['stop_lon'], stop['stop_lat'])),
        axis=1
    )]
    
    # No valid stops found on trajectory
    if stops_on_seg.empty:
        return None, None, None

    # Select closest stop among valid candidates
    closest_stop = stops_on_seg.loc[stops_on_seg['distance'].idxmin()]
    closest_stop_name = closest_stop['stop_name']
    closest_stop_id = closest_stop['stop_id']
    
    # Note: stop_geom stored as (lat, lon) for Folium compatibility
    closest_stop_geom = Point(closest_stop['stop_lat'], closest_stop['stop_lon'])
    
    return closest_stop_name, closest_stop_id, closest_stop_geom

In [ ]:
def gen_seg_data(matched_routes, buffer=0.0005):
    """
    Enrich matched routes with boarding/alighting stop information.
    
    For each matched transit segment, identifies the specific stops where the
    user boarded and alighted by finding GTFS stops near the segment endpoints.
    Also filters out false positives (e.g., very short segments where start
    and end stops are the same).
    
    Parameters
    ----------
    matched_routes : list of dict
        Output from match_all_legs(), containing matched routes
    buffer : float, optional
        Buffer distance in decimal degrees for spatial operations (default: 0.0005)
        ~0.0005° ≈ 50m at mid-latitudes
    
    Returns
    -------
    list of dict
        Enriched segment data with stop information, each containing:
        - seg_geom: GPS trajectory segment
        - route_geom: Matched GTFS route geometry (None if walking)
        - route_name: Transit line name (None if walking)
        - route_id: GTFS route identifier (None if walking)
        - start_stop_name: Boarding stop name (None if walking/not found)
        - start_stop_id: Boarding stop identifier
        - start_stop_geom: Boarding stop location
        - end_stop_name: Alighting stop name (None if walking/not found)
        - end_stop_id: Alighting stop identifier
        - end_stop_geom: Alighting stop location
        
    Notes
    -----
    False positive filtering:
    - If start_stop_id == end_stop_id, the segment is too short to be a real
      transit ride (likely a brief stop or GPS error). These are converted to
      walking segments by setting all route/stop fields to None.
    
    Buffer usage:
    - route_geom.buffer(buffer): Creates tolerance zone around route to account
      for GPS accuracy (~50m)
    - seg_geom.buffer(buffer): Creates tolerance zone around trajectory for
      stop matching
    
    The algorithm queries GTFS data to find:
    1. All trips on the matched route
    2. All stops served by those trips
    3. Stops that lie on the route geometry
    4. Closest stops to segment start/end points
    """
    seg_data = []
    
    for match in matched_routes:
        seg_geom = match['seg_geom']
        route_geom = match['route_geom']
        route_name = match['route_name']
        route_id = match['route_id']
        feed = match['feed']

        # Initialize stop information
        start_stop_name = None
        start_stop_id = None
        start_stop_geom = None
        end_stop_name = None
        end_stop_id = None
        end_stop_geom = None

        # If segment matched to a transit route, identify stops
        if feed:
            trips = feed.trips
            stop_times = feed.stop_times
            stops = feed.stops
        
            # Find all trips on this route
            best_trips = trips[trips['route_id'] == route_id]
            best_trip_ids = best_trips['trip_id'].tolist()
            
            # Find all stop_times for these trips
            best_stop_times = stop_times[stop_times['trip_id'].isin(best_trip_ids)]
            
            # Get stop details
            best_stops = stops[stops['stop_id'].isin(best_stop_times['stop_id'])]
            
            # Filter to stops that lie on the route geometry (with buffer tolerance)
            buffered_route_geom = route_geom.buffer(buffer)
            best_stops_on_route = best_stops[
                best_stops['geometry'].apply(buffered_route_geom.contains)
            ]
            
            if best_stops_on_route.shape[0] > 0:
                # Create buffered segment for stop matching
                buffered_seg_geom = seg_geom.buffer(buffer)
                
                # Get segment start and end coordinates
                start_coords = seg_geom.coords[0]
                end_coords = seg_geom.coords[-1]
            
                # Find closest stops to start and end points
                start_stop_name, start_stop_id, start_stop_geom = find_closest_stop(
                    start_coords, best_stops_on_route, buffered_seg_geom
                )
                end_stop_name, end_stop_id, end_stop_geom = find_closest_stop(
                    end_coords, best_stops_on_route, buffered_seg_geom
                )

            # Filter false positives: segments where boarding and alighting
            # stops are the same (likely not a real transit ride)
            if (start_stop_id == end_stop_id) or (
                start_stop_geom is not None and end_stop_geom is not None and 
                start_stop_geom.buffer(buffer).contains(end_stop_geom)
            ):
                # Convert to walking segment (no route match)
                route_geom = None
                route_name = None
                route_id = None
        
                start_stop_name = None
                start_stop_id = None
                start_stop_geom = None
                end_stop_name = None
                end_stop_id = None
                end_stop_geom = None
        
        # Store enriched segment data
        seg_data.append({
            "seg_geom": seg_geom,
            "route_geom": route_geom,
            "route_name": route_name,
            "route_id": route_id,
            "start_stop_name": start_stop_name,
            "start_stop_id": start_stop_id,
            "start_stop_geom": start_stop_geom,
            "end_stop_name": end_stop_name,
            "end_stop_id": end_stop_id,
            "end_stop_geom": end_stop_geom
        })
        
    return seg_data

In [ ]:
# ============================================================================
# SEGMENT MERGING AND POST-PROCESSING
# ============================================================================

def merge_segments(seg_data):
    """
    Merge consecutive segments on the same transit route.
    
    Sometimes a single real-world transit ride gets split into multiple segments
    due to GPS signal loss, staypoint detection errors, or brief stops that
    create artificial breaks. This function merges consecutive segments that:
    1. Are on the same transit route (same route_id)
    2. Connect at the same stop (end_stop of seg1 == start_stop of seg2)
    
    This reconstruction step ensures each continuous transit ride appears as
    a single segment in the final itinerary.
    
    Parameters
    ----------
    seg_data : list of dict
        Segment data with route and stop information
    
    Returns
    -------
    list of dict
        Merged segments where consecutive segments on the same route are combined
        
    Notes
    -----
    The merging algorithm:
    1. Iterates through segments sequentially
    2. For each segment, checks if it can merge with the next segment
    3. If mergeable, combines their geometries and updates end stop
    4. Continues merging until no more consecutive segments can be merged
    5. Adds the final merged segment to output
    
    Geometry merging:
    - Uses shapely.ops.linemerge to connect LineStrings
    - If linemerge returns MultiLineString (disconnected segments), manually
      concatenates coordinates to force a continuous LineString
    
    Benefits:
    - More accurate representation of user's transit rides
    - Simplifies itinerary (fewer spurious segments)
    - Better calculation of ride duration and distance metrics
    
    Example:
    Before merging:
    - Seg 1: Route 10, Stop A → Stop B
    - Seg 2: Route 10, Stop B → Stop C
    - Seg 3: Route 10, Stop C → Stop D
    
    After merging:
    - Seg 1: Route 10, Stop A → Stop D
    """
    merged_segments = []

    def merge_two_segments(seg1, seg2):
        """Helper function to merge two consecutive segments."""
        seg1_geom = seg1['seg_geom']
        seg2_geom = seg2['seg_geom']

        # Attempt to merge geometries using linemerge
        merged_geom = linemerge([seg1_geom, seg2_geom])
        
        # If linemerge fails to produce single LineString, force concatenation
        if merged_geom.geom_type == "MultiLineString":
            merged_geom = LineString([
                point for line in merged_geom.geoms for point in line.coords
            ])
        
        # Copy seg1 data and update with merged information
        merged_seg = seg1.copy()
        merged_seg['seg_geom'] = merged_geom
        merged_seg['end_stop_name'] = seg2['end_stop_name']
        merged_seg['end_stop_id'] = seg2['end_stop_id']
        merged_seg['end_stop_geom'] = seg2['end_stop_geom']
        
        return merged_seg

    # Iterate through segments and merge where possible
    i = 0
    while i < len(seg_data):
        curr = seg_data[i]

        # Merge with subsequent segments while conditions are met
        while (
            i + 1 < len(seg_data) and
            curr['route_id'] == seg_data[i + 1]['route_id'] and
            curr['end_stop_id'] == seg_data[i + 1]['start_stop_id']
        ):
            curr = merge_two_segments(curr, seg_data[i + 1])
            i += 1

        merged_segments.append(curr)
        i += 1

    return merged_segments

In [ ]:
# ============================================================================
# EXAMPLE USAGE
# ============================================================================

# Step 1: Build R-tree spatial index from GTFS data
# This needs to be done once per session
print("Building R-tree spatial index from GTFS feeds...")
route_index, gtfs_routes = gen_rtree()
print(f"Indexed {len(gtfs_routes)} routes across all transit services")

In [ ]:
# Step 2: Load GPS trajectory data
# Expected columns: tripIdPk, timestamp, lat, lon
legit_trips = pd.read_csv("raw_lat_lons.csv")
legit_trips_tripIdPk = legit_trips['tripIdPk'].unique().tolist()
print(f"Loaded {len(legit_trips_tripIdPk)} unique trips")

In [ ]:
# Step 3: Select a single trip for demonstration
# Replace with any tripIdPk from your dataset
tripIdPk = 'ddb8ae5b-15d5-4906-a504-c6788cda4580'
trip = legit_trips[legit_trips['tripIdPk'] == tripIdPk]
print(f"Processing trip {tripIdPk} with {len(trip)} GPS points")

In [ ]:
# Step 4: Preprocess trajectory (clean, densify, smooth, segment)
print("Preprocessing trajectory...")
pfs = generate_pfs(trip)
print(f"Detected {pfs['tripleg_id'].nunique()} trip legs")

In [ ]:
# Step 5: Match each trip leg to GTFS routes
print("Matching trip legs to transit routes...")
matched_routes = match_all_legs(pfs, gtfs_routes, route_index)
print(f"Matched {sum(1 for m in matched_routes if m['route_name'] is not None)} legs to transit routes")

In [ ]:
# Step 6: Identify boarding/alighting stops
print("Identifying boarding and alighting stops...")
seg_data = gen_seg_data(matched_routes)
print("Stop identification complete")

In [ ]:
# Step 7: Merge consecutive segments on the same route
print("Merging consecutive segments...")
merged = merge_segments(seg_data)
print(f"Final itinerary has {len(merged)} segments")

In [ ]:
# Step 8: Generate summary DataFrame
print("\nReconstructed Itinerary:")
gen_seg_df(merged)

In [ ]:
# Step 9: Visualize reconstructed itinerary on interactive map
print("Generating interactive map...")
plot(merged)